In [10]:

import pandas as pd
import geopandas as gpd
from pathlib import Path
from functools import reduce

In [18]:
# Convert the string to a Path object
extraction_path = Path("/Users/sell/SynologyDrive/Dataset/daina")
region = 'wroclaw'
final_score_out = extraction_path / 'score'/ region / f'result_score_{region}.geojson'

# Now the / operator works perfectly
grid_box_out = extraction_path / 'score' / region / f'grid_box_{region}.geojson'
centroid_box_out = extraction_path / 'score' / region / f'centroid_box_{region}.geojson'
score_box2dso_out = extraction_path / 'score' / region / f'score_box2dso_{region}.geojson'
score_box2solar_out = extraction_path / 'score' / region / f'score_box2solar_{region}.geojson'
score_box2station_out = extraction_path / 'score' / region / f'score_box2station_{region}.geojson'
score_box2road_out = extraction_path / 'score' / region / f'score_box2road_{region}.geojson'
score_dni_out = extraction_path / 'score' / region / f'score_dni_{region}.geojson'
score_pvout_out = extraction_path / 'score' / region / f'score_pvout_{region}.geojson' 
score_temp_out = extraction_path / 'score' / region / f'score_temp_{region}.geojson' 
score_dem_out = extraction_path / 'score' / region / f'score_dem_{region}.geojson'
land_ratio_out = extraction_path / 'score' / region / f'score_landRatio_{region}.geojson'

In [19]:
grid_box = gpd.read_file(grid_box_out)
centroid_box = gpd.read_file(centroid_box_out)
# score centroid box - dso
score_box2dso = gpd.read_file(score_box2dso_out)
score_box2dso.rename(columns={'score':'dso_score'}, inplace = True)
# score centroid box- solar
score_box2solar = gpd.read_file(score_box2solar_out)
score_box2solar.rename(columns={'score':'solar_score'}, inplace = True)
# score centroid box- station
score_box2station = gpd.read_file(score_box2station_out)
score_box2station.rename(columns={'score':'station_score'}, inplace = True)
# score centroid box- road
score_box2road = gpd.read_file(score_box2road_out)
score_box2road.rename(columns={'score':'road_score'}, inplace = True)
# score land ratio
score_box2land = gpd.read_file(land_ratio_out)
score_box2land.rename(columns={'ratio':'land_score'}, inplace = True)
score_box2land_update = score_box2land.groupby('box_id', as_index=False).agg({
    'land_score': 'sum',
    'fclass': lambda x: x.value_counts().index[0]
})

# score dni zonal
score_box2dni = gpd.read_file(score_dni_out)
score_box2dni.rename(columns={'_mean':'dni_score'}, inplace = True)
# score pvout zonal
score_box2pvout = gpd.read_file(score_pvout_out)
score_box2pvout.rename(columns={'_mean':'pvout_score'}, inplace = True)
# score temp zonal
score_box2temp = gpd.read_file(score_temp_out)
score_box2temp.rename(columns={'_mean':'temp_score'}, inplace = True)
# score dem zonal
score_box2dem = gpd.read_file(score_dem_out)
score_box2dem.rename(columns={'_mean':'dem_score'}, inplace = True)


In [20]:

dso = score_box2dso[['box_id','dso_score']]
solar = score_box2solar[['box_id','solar_score']]
station = score_box2station[['box_id','station_score']]
road = score_box2road[['box_id', 'road_score']]
land = score_box2land_update[['box_id','land_score','fclass']]
dni = score_box2dni[['box_id','dni_score']]
pvout = score_box2pvout[['box_id','pvout_score']]
temp = score_box2temp[['box_id','temp_score']]
dem = score_box2dem[['box_id','dem_score']]

In [21]:
# 1. Put all your small dataframes into a list
data_frames = [grid_box, dso, solar, station, road, land, dni, pvout, temp, dem]
# 2. Merge them all sequentially on 'box_id'
# 'how=left' ensures you keep all boxes from your main grid_box
df_merged = reduce(lambda left, right: pd.merge(left, right, on='box_id', how='left'), data_frames)
final_score = df_merged[['box_id', 'dni_score', 'pvout_score', 'temp_score', 'dem_score','dso_score',
       'solar_score', 'station_score', 'road_score', 'land_score','fclass','area', 'perimeter', 'region_name', 'geometry']]


In [22]:
final_score .to_file(final_score_out, driver='GeoJSON')

In [ ]:
# # check the dimension

# scores = [grid_box,centroid_box,score_box2dso, score_box2solar, score_box2station, score_box2road, score_box2dni, score_box2pvout, score_box2temp,score_box2dem, score_box2land]
# score_names = ["grid_box","centroid_box",
#     "score_box2dso", "score_box2solar", "score_box2station", 
#     "score_box2road", "score_box2dni", "score_box2pvout", 
#     "score_box2temp", "score_box2dem", "score_box2land"
# ]

# for i in range(len(scores)):
#     print(score_names[i], scores[i].shape)

In [ ]:
# def get_duplicate_boxes(df, column_name='box_id'):
#     """
#     Returns all rows that share a duplicated ID.
#     """
#     # keep=False marks ALL occurrences of duplicates as True
#     duplicate_mask = df.duplicated(subset=[column_name], keep=False)
    
#     # Filter the dataframe and sort by the ID for easy comparison
#     duplicates_df = df[duplicate_mask].sort_values(by=column_name)
    
#     print(f"Found {duplicates_df[column_name].nunique()} unique IDs that are duplicated.")
#     print(f"Total rows affected: {len(duplicates_df)}")
    
#     return duplicates_df

# ## Usage:
# dup_df = get_duplicate_boxes(score_box2dso)
# display(dup_df)

In [ ]:
# # Convert the string to a Path object
# extraction_path = Path("/Users/sell/SynologyDrive/Dataset/daina")
# region = 'wroclaw'



# def final_score(extraction_path,final_score_out, region):
    
#     # Now the / operator works perfectly
#     grid_box_out = extraction_path / 'score' / region / f'grid_box_{region}.geojson'
#     centroid_box_out = extraction_path / 'score' / region / f'centroid_box_{region}.geojson'
#     score_box2dso_out = extraction_path / 'score' / region / f'score_box2dso_{region}.geojson'
#     score_box2solar_out = extraction_path / 'score' / region / f'score_box2solar_{region}.geojson'
#     score_box2station_out = extraction_path / 'score' / region / f'score_box2station_{region}.geojson'
#     score_box2road_out = extraction_path / 'score' / region / f'score_box2road_{region}.geojson'
#     score_dni_out = extraction_path / 'score' / region / f'score_dni_{region}.geojson'
#     score_pvout_out = extraction_path / 'score' / region / f'score_pvout_{region}.geojson' 
#     score_temp_out = extraction_path / 'score' / region / f'score_temp_{region}.geojson' 
#     score_dem_out = extraction_path / 'score' / region / f'score_dem_{region}.geojson'
#     land_ratio_out = extraction_path / 'score' / region / f'score_landRatio_{region}.geojson'
#     grid_box = gpd.read_file(grid_box_out)
#     centroid_box = gpd.read_file(centroid_box_out)
#     # score centroid box - dso
#     score_box2dso = gpd.read_file(score_box2dso_out)
#     score_box2dso.rename(columns={'score':'dso_score'}, inplace = True)
#     # score centroid box- solar
#     score_box2solar = gpd.read_file(score_box2solar_out)
#     score_box2solar.rename(columns={'score':'solar_score'}, inplace = True)
#     # score centroid box- station
#     score_box2station = gpd.read_file(score_box2station_out)
#     score_box2station.rename(columns={'score':'station_score'}, inplace = True)
#     # score centroid box- road
#     score_box2road = gpd.read_file(score_box2road_out)
#     score_box2road.rename(columns={'score':'road_score'}, inplace = True)
#     # score land ratio
#     score_box2land = gpd.read_file(land_ratio_out)
#     score_box2land.rename(columns={'ratio':'land_score'}, inplace = True)
#     score_box2land_update = score_box2land.groupby('box_id', as_index=False).agg({
#         'land_score': 'sum',
#         'fclass': lambda x: x.value_counts().index[0]
#     })

#     # score dni zonal
#     score_box2dni = gpd.read_file(score_dni_out)
#     score_box2dni.rename(columns={'_mean':'dni_score'}, inplace = True)
#     # score pvout zonal
#     score_box2pvout = gpd.read_file(score_pvout_out)
#     score_box2pvout.rename(columns={'_mean':'pvout_score'}, inplace = True)
#     # score temp zonal
#     score_box2temp = gpd.read_file(score_temp_out)
#     score_box2temp.rename(columns={'_mean':'temp_score'}, inplace = True)
#     # score dem zonal
#     score_box2dem = gpd.read_file(score_dem_out)
#     score_box2dem.rename(columns={'_mean':'dem_score'}, inplace = True)
    
#     dso = score_box2dso[['box_id','dso_score']]
#     solar = score_box2solar[['box_id','solar_score']]
#     station = score_box2station[['box_id','station_score']]
#     road = score_box2road[['box_id', 'road_score']]
#     land = score_box2land_update[['box_id','land_score','fclass']]
#     dni = score_box2dni[['box_id','dni_score']]
#     pvout = score_box2pvout[['box_id','pvout_score']]
#     temp = score_box2temp[['box_id','temp_score']]
#     dem = score_box2dem[['box_id','dem_score']]
    
#     # 1. Put all your small dataframes into a list
#     data_frames = [grid_box, dso, solar, station, road, land, dni, pvout, temp, dem]
#     # 2. Merge them all sequentially on 'box_id'
#     # 'how=left' ensures you keep all boxes from your main grid_box
#     df_merged = reduce(lambda left, right: pd.merge(left, right, on='box_id', how='left'), data_frames)
#     final_score = df_merged[['box_id', 'dni_score', 'pvout_score', 'temp_score', 'dem_score','dso_score',
#         'solar_score', 'station_score', 'road_score', 'land_score','fclass','area', 'perimeter', 'region_name', 'geometry']]
    
#     return final_score .to_file(final_score_out , driver='GeoJSON')
    

In [ ]:
# import pandas as pd
# import geopandas as gpd
# from functools import reduce
# from pathlib import Path

# def final_score(extraction_path, final_score_out, region):
#     """
#     Merges all spatial score layers for a specific region into one GeoJSON.
#     """
#     base_path = Path(extraction_path) / 'score' / region
    
#     # 1. Initialize with main grid geometry
#     grid_path = base_path / f'grid_box_{region}.geojson'
#     if not grid_path.exists():
#         raise FileNotFoundError(f"Base grid not found at {grid_path}")
        
#     grid_box = gpd.read_file(grid_path)
#     grid_box['box_id'] = grid_box['box_id'].astype(str) # Ensure string for merging
#     processed_dfs = [grid_box]

#     # 2. Configuration: {file_suffix: (old_column_name, new_column_name)}
#     # Standard layers (box2...)
#     standard_layers = {
#         'box2dso':     ('score', 'dso_score'),
#         'box2solar':   ('score', 'solar_score'),
#         'box2station': ('score', 'station_score'),
#         'box2road':    ('score', 'road_score'),
#     }
    
#     # Zonal layers (dni, pvout, etc.)
#     zonal_layers = {
#         'dni':   ('_mean', 'dni_score'),
#         'pvout': ('_mean', 'pvout_score'),
#         'temp':  ('_mean', 'temp_score'),
#         'dem':   ('_mean', 'dem_score'),
#     }

#     # 3. Helper to process and append layers
#     def load_and_append(layers_dict):
#         for suffix, (old_col, new_col) in layers_dict.items():
#             f_path = base_path / f'score_{suffix}_{region}.geojson'
#             if f_path.exists():
#                 df = gpd.read_file(f_path)
#                 df['box_id'] = df['box_id'].astype(str)
#                 # Select only necessary columns
#                 df = df[['box_id', old_col]].rename(columns={old_col: new_col})
#                 processed_dfs.append(df)
#             else:
#                 print(f"Warning: File missing -> {f_path.name}")

#     load_and_append(standard_layers)
#     load_and_append(zonal_layers)

#     # 4. Handle Land Ratio (Requires aggregation)
#     land_path = base_path / f'score_landRatio_{region}.geojson'
#     if land_path.exists():
#         land_df = gpd.read_file(land_path)
#         land_df['box_id'] = land_df['box_id'].astype(str)
#         land_agg = land_df.groupby('box_id', as_index=False).agg({
#             'ratio': 'sum',
#             'fclass': lambda x: x.value_counts().index[0]
#         }).rename(columns={'ratio': 'land_score'})
#         processed_dfs.append(land_agg)

#     # 5. Execute Sequential Merge
#     df_merged = reduce(lambda left, right: pd.merge(left, right, on='box_id', how='left'), processed_dfs)

#     # 6. Final Column Ordering
#     cols_order = [
#         'box_id', 'dni_score', 'pvout_score', 'temp_score', 'dem_score',
#         'dso_score', 'solar_score', 'station_score', 'road_score', 
#         'land_score', 'fclass', 'area', 'perimeter', 'region_name', 'geometry'
#     ]
    
#     # Keep only columns that exist in the dataframe
#     final_cols = [c for c in cols_order if c in df_merged.columns]
#     final_df = df_merged[final_cols].copy()

#     # 7. Save to GeoJSON
#     final_df.to_file(final_score_out, driver='GeoJSON')
#     print(f"Done! Final file saved: {final_score_out}")

# # --- Set Paths and Run ---
# extraction_path = Path("/Users/sell/SynologyDrive/Dataset/daina")
# region = 'wroclaw'
# final_score_out = extraction_path / 'score' / region / f'final_score_{region}.geojson'

# final_score(extraction_path, final_score_out, region)

Done! Final file saved: /Users/sell/SynologyDrive/Dataset/daina/score/wroclaw/final_score_wroclaw.geojson


In [ ]:
# def safe_load_geojson(path):
#     """
#     Helper to load GeoJSONs in QGIS environment to avoid 
#     Multi-part geometry NotImplementedError.
#     """
#     try:
#         # Standard load
#         return gpd.read_file(path)
#     except (NotImplementedError, Exception):
#         # Fallback using Fiona for older GeoPandas compatibility
#         with fiona.open(path) as src:
#             return gpd.GeoDataFrame.from_features(src, crs=src.crs)

# def final_score(extraction_path, final_score_out, region):
#     """
#     Merges all spatial score layers into one master GeoJSON.
#     """
#     base_path = Path(extraction_path) / 'score' / region
    
#     # 1. Load the main grid (The file most likely to have MultiPolygons)
#     grid_path = base_path / f'grid_box_{region}.geojson'
#     if not grid_path.exists():
#         raise FileNotFoundError(f"Base grid missing: {grid_path}")
        
#     grid_box = safe_load_geojson(grid_path)
#     grid_box['box_id'] = grid_box['box_id'].astype(str)
#     processed_dfs = [grid_box]

#     # 2. Define standard layers (box2...) and Zonal layers
#     layers_config = {
#         'box2dso':     ('score', 'dso_score'),
#         'box2solar':   ('score', 'solar_score'),
#         'box2station': ('score', 'station_score'),
#         'box2road':    ('score', 'road_score'),
#         'dni':         ('_mean', 'dni_score'),
#         'pvout':       ('_mean', 'pvout_score'),
#         'temp':        ('_mean', 'temp_score'),
#         'dem':         ('_mean', 'dem_score')
#     }

#     # 3. Process layers
#     for suffix, (old_col, new_col) in layers_config.items():
#         f_path = base_path / f'score_{suffix}_{region}.geojson'
#         if f_path.exists():
#             # Use safe loader to prevent geometry errors
#             df = safe_load_geojson(f_path)
#             df['box_id'] = df['box_id'].astype(str)
#             # We only need the ID and the score value for the merge
#             processed_dfs.append(df[['box_id', old_col]].rename(columns={old_col: new_col}))
#         else:
#             print(f"Warning: Layer {suffix} not found.")

#     # 4. Handle Land Ratio aggregation
#     land_path = base_path / f'score_landRatio_{region}.geojson'
#     if land_path.exists():
#         land_df = safe_load_geojson(land_path)
#         land_df['box_id'] = land_df['box_id'].astype(str)
#         land_agg = land_df.groupby('box_id', as_index=False).agg({
#             'ratio': 'sum',
#             'fclass': lambda x: x.value_counts().index[0] if not x.empty else None
#         }).rename(columns={'ratio': 'land_score'})
#         processed_dfs.append(land_agg)

#     # 5. Merge all DataFrames
#     df_merged = reduce(lambda left, right: pd.merge(left, right, on='box_id', how='left'), processed_dfs)

#     # 6. Final Column Selection
#     cols_order = [
#         'box_id', 'dni_score', 'pvout_score', 'temp_score', 'dem_score',
#         'dso_score', 'solar_score', 'station_score', 'road_score', 
#         'land_score', 'fclass', 'area', 'perimeter', 'region_name', 'geometry'
#     ]
    
#     final_cols = [c for c in cols_order if c in df_merged.columns]
#     final_df = df_merged[final_cols].copy()

#     # 7. Save result
#     final_df.to_file(final_score_out, driver='GeoJSON')
#     print(f"Success! Final score saved to {final_score_out.name}")

In [23]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
from pymcdm.methods import TOPSIS

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
from pymcdm.methods import TOPSIS

def mcdm(input_path):
    # 1. LOAD GEOJSON DATA
    gdf = gpd.read_file(input_path)

    # 2. PREPROCESSING
    # Clean up columns not needed for the math
    cols_to_drop = ['perimeter', 'region_name', 'area', 'fclass']
    gdf_reduced = gdf.drop(columns=cols_to_drop, errors='ignore')

    # 3. CONSTRUCT THE DECISION MATRIX
    # Ensure criteria_cols matches the data in your specific GeoJSON
    criteria_cols = [
        'dni_score', 'temp_score', 'pvout_score', 'dem_score', 
        'road_score', 'station_score', 'solar_score', 'dso_score', 'land_score'
    ]

    # Drop rows with missing values in these specific columns
    gdf_clean = gdf_reduced.dropna(subset=criteria_cols).copy()
    matrix = gdf_clean[criteria_cols].values

    # 4. DEFINE WEIGHTS & TYPES (Updated to 9 elements)
    # We must have exactly 9 weights to match the 9 criteria above
    weights = np.array([1/9] * 9) 

    # Types: 1 for Benefit (Higher is better), -1 for Cost (Lower is better)
    # Mapping based on typical solar MCDM logic:
    # dni(1), temp(-1), pvout(1), dem(-1), road(1), station(1), solar(1), dso(1), land(1)
    types = np.array([1, -1, 1, -1, 1, 1, 1, 1, 1]) 

    # 5. RUN THE MCDM MODEL (TOPSIS)
    topsis = TOPSIS()
    pref = topsis(matrix, weights, types)

    # Attach results
    gdf_clean['rank_score'] = pref
    gdf_clean['rank'] = gdf_clean['rank_score'].rank(ascending=False)

    # 6. SPATIAL VISUALIZATION (Adjusted for Wrocław)
    top_sites = gdf_clean.nsmallest(50, 'rank')
    top_sites.to_file(output_path, driver='GeoJSON')

    # fig = px.choropleth_mapbox(
    #     top_sites,
    #     geojson=top_sites.geometry.__geo_interface__, # Standardize geometry for Plotly
    #     locations=top_sites.index,
    #     color='rank_score',
    #     center={"lat": 51.1079, "lon": 17.0385}, # Centered specifically on Wrocław
    #     mapbox_style="carto-positron",
    #     zoom=10,
    #     title="Top 50 Solar Sites in Wrocław (MCDM Ranking)"
    # )
    # fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    # fig.show()

/var/folders/kt/6_c8zc9d1w18l8r74nq1kc380000gn/T/ipykernel_6889/1091446207.py:46: DeprecationWarning: *choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.choropleth_mapbox(


In [27]:
mcdm = '/Users/sell/SynologyDrive/Dataset/daina/score/wroclaw/mcdm_wroclaw.geojson'
top_sites.to_file(mcdm, driver='GeoJSON')

In [28]:
df = pd.read_csv('/Users/sell/Downloads/mcdm_multi.csv')

In [29]:
df

,osm_id,topsis_score,waspas_score,vikor_score,rank_topsis,rank_waspas,rank_vikor
0,676612,0.367987,0.207435,0.743216,143015,167475,135902
1,676611,0.367889,0.207434,0.743236,143084,167476,135906
2,676610,0.367683,0.207285,0.743512,143254,167489,135963
3,676607,0.236948,0.149614,0.958054,169105,169124,169064
4,676608,0.266047,0.168955,0.826319,168804,168964,158517
...,...,...,...,...,...,...,...
169123,17025,0.357418,0.356181,0.676120,150463,107021,120330
169124,17024,0.372977,0.368183,0.664582,139032,104733,117038
169125,17023,0.369035,0.365536,0.666691,142203,105268,117646
169126,17022,0.403396,0.389285,0.643445,116314,100415,109856
